In [149]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse

In [150]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [151]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [152]:
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

search_button.click()

In [153]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword
keyword = "@fruttistudio"
searchbox.send_keys(keyword)

In [154]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Remove the "@" symbol
    keyword = keyword[1:]
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')
first_result.click()

In [142]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(3)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration

In [143]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all image elements that match the specific class in the current soup
    elements = soup.find_all('a', class_='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')

    # Extract the href attributes and filter URLs that start with "/p/" or "/p/CZK8AFkFGRa/"
    post_urls.extend([element['href'] for element in elements if element['href'].startswith('/p/')])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

unique_post_urls

['/p/CKBjkihl2nk/',
 '/p/CRJyHXYtnXV/',
 '/p/CKJv5uSFrvi/',
 '/p/CKRBNiclu7P/',
 '/p/CJuNfpvgrQf/',
 '/p/CRQ-R_tpuqA/',
 '/p/CRMxxTvHQkn/',
 '/p/CNuwNUdsil3/',
 '/p/CKj1_TPghPn/',
 '/p/CKOjkTYA12t/',
 '/p/CKEJ59mFuqg/',
 '/p/CMXXKIjlo0o/',
 '/p/CJ--S7Cl9qH/',
 '/p/CRTsDEvn22a/',
 '/p/CNpUawYIcbe/',
 '/p/CLhAWcwl4TB/',
 '/p/CJuNwZ4AMr8/',
 '/p/CK6XmPKFSsy/',
 '/p/CN96sSHoBSF/',
 '/p/CK_xacWlXVg/',
 '/p/CODNE9IsG1Z/',
 '/p/CJuOPnQgWer/',
 '/p/COiR6runHxd/',
 '/p/CJ3nrqjAP5V/',
 '/p/CJ1GUU8g9x9/',
 '/p/CJuN71tAL8K/',
 '/p/CJuMPRvgexk/',
 '/p/CROrdSSJ91S/',
 '/p/CJuNL1VAP1l/',
 '/p/CN2kHDas3wg/',
 '/p/CNm73-iM128/',
 '/p/CLO9ZASFnkJ/',
 '/p/CJuNAfUAqkK/',
 '/p/CN4xIsoHnEo/',
 '/p/CNXhHqkh4rG/',
 '/p/CJuOzTyA86t/',
 '/p/CMSGDaXFUgR/',
 '/p/CReImpzKIiM/',
 '/p/CLwue2DF3Cy/',
 '/p/CKoLF9DllZy/',
 '/p/CNdkeB2Hoas/',
 '/p/CLmlyNslzZA/',
 '/p/CLbzoUDFPC4/',
 '/p/CLEjA2BlEx3/',
 '/p/CNkKwOuI235/',
 '/p/CKWPB86FI48/',
 '/p/CJ8Y2PYgxCb/',
 '/p/CNccMRFH4Mr/',
 '/p/COIPydLhR-_/',
 '/p/CONoDsMBrLM/',


In [144]:
json_list = []

# Define the query parameters to add
query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:

    # Get the current URL of the page
    current_url = driver.current_url

    # Append the query parameters to the current URL
    modified_url = "https://www.instagram.com/" + url + "?" + query_parameters
    
    #Get URL
    
    driver.get(modified_url)
    
    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(1)
    
    # Find the <pre> tag containing the JSON data
    pre_tag = driver.find_element_by_xpath('//pre')

    # Extract the JSON data from the <pre> tag
    json_script = pre_tag.text

    # Parse the JSON data
    json_parsed = json.loads(json_script)
    
    # Add json to the list
    json_list.append(json_parsed)
    
driver.quit()


C:\Users\Victor\AppData\Local\Temp\ipykernel_6848\4264131524.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pre_tag = driver.find_element_by_xpath('//pre')


In [145]:
carousel_list = []

for json in json_list:

    # Extract the list from the 'items' key
    item_list = json.get('items', [])

    # Extract the list from the 'carousel_media' key
    carousel_media = item_list[0].get('carousel_media', [])
    
    # Handle cases of unique image, instead of carousel    
    unique_media = item_list[0].get('image_versions2', [])
    
    carousel_list.append(carousel_media)

In [ ]:
media_list = []

for json in json_list:
    # Extract the list from the 'items' key
    item_list = json.get('items', [])

    # Check if 'carousel_media' is present in the first item
    if 'carousel_media' in item_list[0]:
        carousel_media = item_list[0]['carousel_media']
        media_list.append(carousel_media)
    elif 'image_versions2' in item_list[0]:
        # Handle cases of unique image, instead of carousel
        unique_media = item_list[0]['image_versions2']
        media_list.append([unique_media])
    else:
        # Handle cases where neither 'carousel_media' nor 'image_versions2' is present
        media_list.append([])

In [146]:
all_urls = []

for carousel_media in carousel_list:

    for media in carousel_media:
        image_url = media['image_versions2']['candidates'][0]['url']
        all_urls.append(image_url)

        # Check if 'video_versions' key exists before accessing it
        if 'video_versions' in media:
            video_url = media['video_versions'][0]['url']
            all_urls.append(video_url)

# Print or use all collected URLs as needed
all_urls

['https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/214363416_881183039417684_1142660089053617626_n.jpg?stp=dst-jpg_e15&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDEwODAuc2RyIn0&_nc_ht=scontent-yyz1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=8PbcWmRCpq0AX-WHGaT&edm=AABBvjUBAAAA&ccb=7-5&ig_cache_key=MjYxNTY4NDM3MzUxMDE5MDE3NA%3D%3D.2-ccb7-5&oh=00_AfA2euapxHetSWbDUN5mp4nCa_qtCTah8UDpuDXnSflTZw&oe=65750B16&_nc_sid=4f4799',
 'https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/213804702_217682356879204_3044994935440581477_n.jpg?stp=dst-jpg_e15&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDEwNzkuc2RyIn0&_nc_ht=scontent-yyz1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=aWQ4K34L3D8AX-a_1Z6&edm=AABBvjUBAAAA&ccb=7-5&ig_cache_key=MjYxNTY4NDM3MzM5MjgxMTg0Mg%3D%3D.2-ccb7-5&oh=00_AfAIi8WWONjaJ5nL0SArMkns1volurgsoZQeKXMYl6htog&oe=6574DF46&_nc_sid=4f4799',
 'https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/217640273_2054847527990366_4984269139995702972_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5j

In [159]:
all_urls = []

for json in json_list:
    # Extract the list from the 'items' key
    item_list = json.get('items', [])

    # Check if 'carousel_media' is present in the first item
    if item_list:
        first_item = item_list[0]

        if 'carousel_media' in first_item:
            carousel_media = first_item['carousel_media']
            for media in carousel_media:
                image_url = media.get('image_versions2', {}).get('candidates', [{}])[0].get('url', None)
                if image_url:
                    all_urls.append(image_url)

                # Check if 'video_versions' key exists inside 'image_versions2'
                if 'image_versions2' in media and 'video_versions' in media['image_versions2']:
                    video_url = media['video_versions'][0].get('url', None)
                    if video_url:
                        all_urls.append(video_url)

        elif 'image_versions2' in first_item:
            # Handle cases of unique image, instead of carousel
            unique_media = first_item['image_versions2']
            image_url = unique_media.get('candidates', [{}])[0].get('url', None)
            if image_url:
                all_urls.append(image_url)

            # Check if 'video_versions' key exists inside 'image_versions2'
            if 'video_versions' in unique_media:
                video_url = unique_media['video_versions'][0].get('url', None)
                if video_url:
                    all_urls.append(video_url)

# Print or use all collected URLs as needed
all_urls

['https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/139372676_2798366143757915_228751658551276941_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE0Mzguc2RyIn0&_nc_ht=scontent-yyz1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=j5a4gMry1ioAX8zgCc_&edm=AABBvjUBAAAA&ccb=7-5&ig_cache_key=MjQ4NjQyNDkxMTg4NTA2ODc3Mg%3D%3D.2-ccb7-5&oh=00_AfCZ7FLyxf4govKcY_9eSTi5cQ84pUwqIa5_qnnT2iBRUQ&oe=65749D38&_nc_sid=4f4799',
 'https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/212722888_540322103769056_5667810726820787229_n.jpg?stp=dst-jpg_e15&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDEwNzkuc2RyIn0&_nc_ht=scontent-yyz1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=GVUsvErXSpEAX_7iz4P&edm=AABBvjUBAAAA&ccb=7-5&ig_cache_key=MjYxNDg0MTQ2NzEzNzg0ODc4OQ%3D%3D.2-ccb7-5&oh=00_AfDFhgd3ZbSah4mnT-xKJgfO0wP-CUzpFG3CKaeZ70FH_g&oe=6573F7D7&_nc_sid=4f4799',
 'https://scontent-yyz1-1.cdninstagram.com/v/t51.2885-15/139435133_414431513218160_3434479252040678103_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb

In [160]:
# Create a directory to store downloaded files
download_dir = keyword
os.makedirs(download_dir, exist_ok=True)

# Iterate through all URLs in the all_urls list and download media
for index, url in enumerate(all_urls, 1):
   
    response = requests.get(url, stream=True)

    # Extract file extension from the URL
    url_path = urlparse(url).path
    file_extension = os.path.splitext(url_path)[1]

    # Determine the file name based on the URL
    file_name = f"{index}{file_extension}"

    # Save the file to the download directory
    file_path = os.path.join(download_dir, file_name)

    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"Downloaded: {file_path}")

print(f"Downloaded {len(all_urls)} files to {download_dir}")

Downloaded: fruttistudio\1.jpg
Downloaded: fruttistudio\2.jpg
Downloaded: fruttistudio\3.jpg
Downloaded: fruttistudio\4.jpg
Downloaded: fruttistudio\5.jpg
Downloaded: fruttistudio\6.jpg
Downloaded: fruttistudio\7.jpg
Downloaded: fruttistudio\8.jpg
Downloaded: fruttistudio\9.jpg
Downloaded: fruttistudio\10.jpg
Downloaded: fruttistudio\11.jpg
Downloaded: fruttistudio\12.jpg
Downloaded: fruttistudio\13.jpg
Downloaded: fruttistudio\14.jpg
Downloaded: fruttistudio\15.jpg
Downloaded: fruttistudio\16.jpg
Downloaded: fruttistudio\17.jpg
Downloaded: fruttistudio\18.jpg
Downloaded: fruttistudio\19.jpg
Downloaded: fruttistudio\20.jpg
Downloaded: fruttistudio\21.jpg
Downloaded: fruttistudio\22.jpg
Downloaded: fruttistudio\23.jpg
Downloaded: fruttistudio\24.jpg
Downloaded: fruttistudio\25.jpg
Downloaded: fruttistudio\26.jpg
Downloaded: fruttistudio\27.jpg
Downloaded: fruttistudio\28.jpg
Downloaded: fruttistudio\29.jpg
Downloaded: fruttistudio\30.jpg
Downloaded: fruttistudio\31.jpg
Downloaded: frutt